In [2]:
import re

def get_cpu_model_name():
    """
    Manually parses /proc/cpuinfo to find the 'model name'.
    Returns a list of all model names found (one for each logical core).
    """
    model_names = []
    try:
        with open('/proc/cpuinfo', 'r') as f:
            for line in f:
                if 'model name' in line:
                    # Use regex to extract the text after the colon and strip whitespace
                    match = re.search(r'model name\s*:\s*(.*)', line)
                    if match:
                        model_names.append(match.group(1).strip())
    except IOError:
        print("Error: Could not read /proc/cpuinfo file.")
        return []
    
    # Return unique model names if there are multiple identical cores
    return list(set(model_names))

if __name__ == '__main__':
    cpu_models = get_cpu_model_name()
    if cpu_models:
        for index, model in enumerate(cpu_models):
            print(f"CPU Model {index + 1}: {model}")
    else:
        print("Could not determine CPU model name.")


CPU Model 1: AMD Ryzen 9 9950X3D 16-Core Processor


In [3]:
cpu_models[0]

'AMD Ryzen 9 9950X3D 16-Core Processor'

In [4]:
import platform
import psutil
import json

def get_system_info():
    info = {}
    info['Plataforma'] = platform.system()
    #info['Nome do Nó'] = platform.node()
    info[f'Versão do SO'] = platform.release()
    info['Arquitetura'] = platform.machine()
    info['Processador'] = cpu_models[0]
    # Informação de RAM em GB
    info['RAM Total'] = str(round(psutil.virtual_memory().total / (1024.0 ** 3))) + " GB"
    # Informação da CPU (exemplo simples)
    info['CPU Cores (físicos)'] = psutil.cpu_count(logical=False)
    info['CPU Cores (lógicos)'] = psutil.cpu_count(logical=True)
    return info
print("="*40, "Informações do Sistema e Hardware", "="*40)
#print(get_system_info())
for key, value in get_system_info().items():
    print(f"{key}: {value}")
print("="*40, "="*40)



======================================== Informações do Sistema e Hardware ========================================
Plataforma: Linux
Versão do SO: 6.12.63-1-MANJARO
Arquitetura: x86_64
Processador: AMD Ryzen 9 9950X3D 16-Core Processor
RAM Total: 60 GB
CPU Cores (físicos): 16
CPU Cores (lógicos): 32
======================================== ========================================


In [5]:
import torch

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version PyTorch was built with: {torch.version.cuda}")

print(f"Python version: {platform.python_version()}")
print(f"PyTorch version: {torch.__version__}")
#print(f"CUDA available: {torch.cuda.is_available()}")


GPU Name: NVIDIA GeForce RTX 5070 Ti
CUDA version PyTorch was built with: 12.8
Python version: 3.11.14
PyTorch version: 2.11.0.dev20260108+cu128


## Gerador de voz

In [7]:
import numpy as np
import sounddevice as sd
from piper.voice import PiperVoice

# 1. Carregar o modelo
model_path = "voz_br.onnx"
config_path = "voz_br.onnx.json"
voice = PiperVoice.load(model_path, config_path)

def sintetizar_voz_br(texto):
    print("🗣️ Sintetizando áudio (Piper PT-BR)...")
    
    audio_segments = []
    
    # Iteramos sobre os chunks gerados pelo Piper
    for chunk in voice.synthesize(texto):
        # Baseado no seu dir(chunk), o atributo correto é audio_int16_array
        if hasattr(chunk, 'audio_int16_array'):
            audio_segments.append(chunk.audio_int16_array)
    
    if not audio_segments:
        print("❌ Nenhum dado de áudio foi gerado.")
        return

    # Concatena os arrays NumPy
    full_audio = np.concatenate(audio_segments)
    
    # Normaliza int16 -> float32 para o sounddevice (divisão por 32768)
    audio_float = full_audio.astype(np.float32) / 32768.0
    
    # Taxa de amostragem padrão (22050Hz para o modelo Faber)
    rate = 22050 
    
    print(f"🔊 Reproduzindo na sua RTX 5070 Ti...")
    sd.play(audio_float, samplerate=rate)
    sd.wait()
    print("✅ Concluído!")

# Teste final:
sintetizar_voz_br("O oráculo está vivo e falando português brasileiro perfeitamente!")

🗣️ Sintetizando áudio (Piper PT-BR)...
🔊 Reproduzindo na sua RTX 5070 Ti...
✅ Concluído!


## Gravação de audio

In [8]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import tempfile
import os

def gravar_audio(duracao_segundos=5, taxa_amostragem=16000):

    """
    Grava áudio do microfone e salva em um arquivo temporário.
    Grava áudio do microfone e salva em um arquivo temporário.
    O Whisper funciona melhor com 16000Hz.
    """
    print(f"🎤 Ouvindo por {duracao_segundos} segundos... Fale agora!")

    # Captura o áudio (mono, float32)

    gravacao = sd.rec(
    int(duracao_segundos * taxa_amostragem),
    samplerate=taxa_amostragem,
    channels=1,
    dtype='float32'
    )

    # Aguarda a gravação terminar

    sd.wait()
    print("✅ Gravação finalizada.")

    # Salva em um arquivo temporário para o Whisper ler

    temp_file = tempfile.mktemp(suffix='.wav')
    write(temp_file, taxa_amostragem, gravacao)

    return temp_file

# Teste rápido de gravação:

arquivo_teste = gravar_audio(duracao_segundos=4)

🎤 Ouvindo por 4 segundos... Fale agora!
✅ Gravação finalizada.


In [9]:
arquivo_teste

'/tmp/tmpoc1oh3iw.wav'

In [10]:
from IPython.display import Audio, display

# Supondo que 'arquivo_teste' seja o caminho do arquivo .wav gerado
print(f"Ouvindo o arquivo: {arquivo_teste}")

# O player do Jupyter carrega o arquivo do disco
display(Audio(arquivo_teste))

Ouvindo o arquivo: /tmp/tmpoc1oh3iw.wav


## Transcrição de audio para fala

In [15]:
from faster_whisper import WhisperModel
import os

# 1. Inicialização do modelo (se ainda não o fez nesta sessão)
# Para a série 50, o flash_attention pode acelerar ainda mais se disponível
print("🚀 Carregando modelo na RTX 5070 Ti...")
model_stt = WhisperModel(
    "medium", 
    device="cuda", 
    compute_type="float16"
)

def transcrever_audio(caminho_arquivo):
    if not os.path.exists(caminho_arquivo):
        print("❌ Arquivo não encontrado!")
        return ""

    print("🔍 Iniciando transcrição...")
    
    # Parâmetros vitais para evitar transcrições vazias:
    # - language="pt": Evita que ele perca tempo tentando identificar o idioma.
    # - vad_filter=True: Remove silêncios brancos que podem confundir o modelo.
    # - beam_size=5: Melhora a precisão da busca.
    segments, info = model_stt.transcribe(
        caminho_arquivo, 
        beam_size=5,
        language="pt",
        vad_filter=True,
        vad_parameters=dict(min_silence_duration_ms=500)
    )

    print(f"Detectado idioma: {info.language} (Probabilidade: {info.language_probability:.2f})")

    texto_final = ""
    # Itera pelos segmentos e imprime conforme aparecem
    for segment in segments:
        # CORREÇÃO: Usamos .2f para float. O 's' de segundos deve ficar fora das chaves.
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
        texto_final += segment.text + " "

    return texto_final.strip()


🚀 Carregando modelo na RTX 5070 Ti...


In [16]:
# --- EXECUÇÃO ---
resultado = transcrever_audio(arquivo_teste)

if resultado:
    print(f"\n✅ TRANSCRITO: {resultado}")
else:
    print("\n⚠️ O modelo não detectou fala no áudio. Verifique o volume da gravação.")

🔍 Iniciando transcrição...
Detectado idioma: pt (Probabilidade: 1.00)
[0.37s -> 2.37s]  Teste de gravação de áudio.

✅ TRANSCRITO: Teste de gravação de áudio.


## Testando o modelo de LLM 

In [20]:
import ollama

def testar_gemma(texto_usuario):
    try:
        response = ollama.chat(
            model='gemma3:12b-it-qat', 
            messages=[
                {'role': 'system', 'content': 'Você é um assistente conciso. Responda apenas com texto puro.'},
                {'role': 'user', 'content': texto_usuario},
            ]
        )
        
        resposta_texto = response['message']['content']
        
        # LIMPEZA: Remove tokens de parada e espaços extras
        tokens_para_remover = ["</end_of_turn>", "<pad>", "<eos>"]
        for token in tokens_para_remover:
            resposta_texto = resposta_texto.replace(token, "")
            
        resposta_texto = resposta_texto.strip()
        
        print(f"🤖 Oráculo: {resposta_texto}")
        return resposta_texto

    except Exception as e:
        print(f"❌ Erro: {e}")
        return "Erro na inteligência."

# Teste manual
resposta = testar_gemma("Quem é você e qual placa de vídeo você está usando?")

🤖 Oráculo: Sou um modelo de linguagem grande, treinado pelo Google. Não uso placas de vídeo.


## Rodando o teste final    

In [21]:
# 1. Grava e Transcreve (o que fizemos antes)
arquivo_audio = gravar_audio(duracao_segundos=5)
texto_pergunta = transcrever_audio(arquivo_audio)

if texto_pergunta:
    # 2. Envia para a LLM
    resposta_ia = testar_gemma(texto_pergunta)
    
    # 3. Faz a IA falar (usando a função do Piper que validamos)
    sintetizar_voz_br(resposta_ia)
else:
    print("Nenhum texto detectado para enviar à LLM.")

🎤 Ouvindo por 5 segundos... Fale agora!
✅ Gravação finalizada.
🔍 Iniciando transcrição...
Detectado idioma: pt (Probabilidade: 1.00)
[0.18s -> 3.18s]  Qual a melhor forma de estudar a programação?
🤖 Oráculo: Prática, projetos, comunidade, recursos online.
🗣️ Sintetizando áudio (Piper PT-BR)...
🔊 Reproduzindo na sua RTX 5070 Ti...
✅ Concluído!
